In [1]:
# import sys
# sys.path # python导入package时自动搜索的路径
# sys.path.append("/root/Docqa")
import sys
sys.path
sys.path.append("/root/")  
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import PyPDFLoader, ArxivLoader,PyPDFDirectoryLoader
from Docqa.text_spliter import ChineseTextSplitter
import PyPDF2 # pdf reader
from pypdf import PdfReader
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationalRetrievalChain


In [2]:

loader = Docx2txtLoader('/data/documents/分布式计算系统.docx')
input_doc = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=650, chunk_overlap=0)
texts = text_splitter.split_documents(input_doc)

In [3]:

loader = PyPDFLoader("/data/documents/深度学习_582660_231113.pdf")
pages = loader.load()
pdf_splitter = ChineseTextSplitter(chunk_size=650, chunk_overlap=0, pdf=True)
pdf_text = pdf_splitter.split_documents(pages)


In [2]:
loader = PyPDFLoader("/data/documents/深度学习_582660_231113.pdf")
pages = loader.load()
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=165)
pdf_text = pdf_splitter.split_documents(pages)
len(pdf_text)

6

In [3]:
from bigdl.llm.langchain.embeddings import TransformersEmbeddings

embeddings = TransformersEmbeddings.from_model_id(model_id="/data/chatglm3-6B/")

/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-21 07:27:30,256 - WARNING - BigdlNativeEmbeddings has been deprecated, please switch to the new LMEmbeddings API for sepcific models.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ChatGLMTokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transfor

In [4]:
from langchain.vectorstores import FAISS

vectorstore_db = FAISS.from_documents(pdf_text, embeddings)
retriever = vectorstore_db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

2024-01-21 07:28:20,356 - INFO - Loading faiss with AVX2 support.
2024-01-21 07:28:20,388 - INFO - Successfully loaded faiss with AVX2 support.


In [5]:
from langchain.prompts import PromptTemplate
custom_prompt_template = """
You are an assistant for question-answering tasks. Use the retrieved {context} and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
History = {history}
Question= {question}
Don't add text above in the result.
Helpful Answer:
"""

prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=["question", "context", "history"])

In [6]:
from langchain.chains.chat_vector_db.prompts import QA_PROMPT
from langchain.chains.question_answering import load_qa_chain

from bigdl.llm.langchain.llms import TransformersLLM

llm = TransformersLLM.from_model_id(
        model_id="/data/chatglm3-6B/",
        model_kwargs={"temperature": 0, "max_length": 4096, "trust_remote_code": True},
    )

2024-01-21 07:28:27,771 - WARNING - BigdlNativeLLM has been deprecated, please switch to the new LLM API for sepcific models.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ChatGLMTokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type chatglm3 to instantiate a model of type chatglm. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████| 7/7 [00:07<00:00,  1.13s/it]
2024-01-21 07:28:35,847 - INFO - Converting the current model to sym_int4 format......


In [7]:
qa_chain_with_memory = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',
                                                   retriever = vectorstore_db.as_retriever(),
                                                   return_source_documents = False,
                                                   chain_type_kwargs = {"verbose": False,
                                                                        "prompt": prompt,
                                                                        "memory": ConversationBufferMemory(
                                                                            input_key="question",
                                                                            memory_key="history",
                                                                            return_messages=False)})

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# qa_chain_with_memory = ConversationalRetrievalChain.from_llm(llm=llm, chain_type='stuff', retriever=retriever, memory=memory)


qqqqqqqqqqq
context


In [8]:

query = "这篇文章提出了什么方法?"
qa_chain_with_memory({"query": query})
# ret = qa_chain_with_memory({"question": query})

/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


pppppppppppp


/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


文章提出了基于深度学习的文章摘要生成方法。该方法通过训练深度神经网络来学习文章的特征表示，并使用该表示来生成摘要。



-------------------------
文章提出了基于深度学习的文章摘要生成方法。该方法通过训练深度神经网络来学习文章的特征表示，并使用该表示来生成摘要。

 

++++++++++++++
文章提出了基于深度学习的文章摘要生成方法。该方法通过训练深度神经网络来学习文章的特征表示，并使用该表示来生成摘要。

 



{'query': '这篇文章提出了什么方法?',
 'result': '文章提出了基于深度学习的文章摘要生成方法。该方法通过训练深度神经网络来学习文章的特征表示，并使用该表示来生成摘要。\n\n \n'}

In [9]:
query = "介绍一下论文使用的数据集"
ret = qa_chain_with_memory({"question": query})

/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


文章使用的数据集是什么？


: 

In [14]:
query = "simply introduce the data sets the paper used."
qa_chain_with_memory({"query": query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= are often correct, thus thresholding ﬁlters out erroneous low-
conﬁdence predictions, leading to a boost in the sign accu-
racy.
Another highlight is the signiﬁcant gap that CTTS has
achieved for thresholded 3-class∗accuracy compared to
other baselines. This can be harnessed in the future for trad-
ing decision-making. For example, a straightforward trad-
ing decision can be buy/sell/hold stocks when the predicted
class is up/down/ﬂat, respectively. Given that CTTS’s pre-
dicted probablitlies are reliable as discussed earlier, the
amount of stock shares to buy/sell/hold can d

/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The paper used licensed Bloomberg data service for intraday stock prices of S&P 500 constituents for the year 2019, sampled at 1-minute interval for 52 weeks with 7 time series for each week. The data was randomly split into training (80%) and validation (20%) sets.

> Finished chain.

> Finished chain.


{'query': 'simply introduce the data sets the paper used.',
 'result': 'You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.\nIf you don\'t know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.\nContext= are often correct, thus thresholding ﬁlters out erroneous low-\nconﬁdence predictions, leading to a boost in the sign accu-\nracy.\nAnother highlight is the signiﬁcant gap that CTTS has\nachieved for thresholded 3-class∗accuracy compared to\nother baselines. This can be harnessed in the future for trad-\ning decision-making. For example, a straightforward trad-\ning decision can be buy/sell/hold stocks when the predicted\nclass is up/down/ﬂat, respectively. Given that CTTS’s pre-\ndicted probablitlies are reliable as discussed earlier, the\namount of stock shares to buy/sell/hold can depend on the\n\nand 4

In [15]:
query = "summarize the paper, please."
qa_chain_with_memory({"query": query})

Token indices sequence length is longer than the specified maximum sequence length for this model (4873 > 4096). Running this sequence through the model will result in indexing errors
/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 4873, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= work, autoregression and recurrent neural networks (Hastie,
Tibshirani, and Friedman 2001). More recent works have ap-
plied Deep learning (DL) methods on numeric time series
data (Bao, Yue, and Rao 2017; Gensler et al. 2016; Romeu
et al. 2015; Sagheer and Kotb 2019; Sutskever, Vinyals, and
Le 2014). DL automates the process of feature extraction
and eliminates the need for domain expertise.
Since the introduction of transformers (Vaswani et al.
2017), they have become the state of the art model to im-arXiv:2304.04912v1  [cs.LG]  11 Apr 2023

and long-term dependencies within

{'query': 'summarize the paper, please.',
 'result': 'You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.\nIf you don\'t know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.\nContext= work, autoregression and recurrent neural networks (Hastie,\nTibshirani, and Friedman 2001). More recent works have ap-\nplied Deep learning (DL) methods on numeric time series\ndata (Bao, Yue, and Rao 2017; Gensler et al. 2016; Romeu\net al. 2015; Sagheer and Kotb 2019; Sutskever, Vinyals, and\nLe 2014). DL automates the process of feature extraction\nand eliminates the need for domain expertise.\nSince the introduction of transformers (Vaswani et al.\n2017), they have become the state of the art model to im-arXiv:2304.04912v1  [cs.LG]  11 Apr 2023\n\nand long-term dependencies within a time series, and forecast\nif the pr

In [7]:
# query = "这篇文章的主题是什么\n"
# docs = docsearch.get_relevant_documents(query)
# print("-"*20+"number of relevant documents"+"-"*20)
# print(len(docs))

--------------------number of relevant documents--------------------
4


In [19]:
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.schema.output import LLMResult

class ChainStreamHandler(CallbackManagerForLLMRun):
    def __init__(self):
        self.tokens = []
        self.num = 0
        # 记得结束后这里置true
        self.finish = False

    def on_llm_new_token(self, token: str, **kwargs):
        print(token)
        self.num += 1
        self.tokens.append(token)

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        self.finish = 1

    def on_llm_error(self, error: Exception, **kwargs) -> None:
        print(str(error))
        self.tokens.append(str(error))

    def generate_tokens(self):
        while not self.finish or self.tokens:
            if self.tokens:
                data = self.tokens.pop(0)
                yield data
            else:
                pass

In [20]:
mycallbackmanager = ChainStreamHandler()

In [24]:
llm_stream = TransformersLLM.from_model_id(
        model_id="/data/chatglm3-6B/",
        model_kwargs={"temperature": 0, "max_length": 4096, "trust_remote_code": True},
        run_manager = mycallbackmanager
        
    )

AttributeError: 'ChainStreamHandler' object has no attribute 'dict'

In [2]:
from transformers import AutoTokenizer, LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained('/data/chatglm3-6B/')

/root/miniconda3/envs/llm-tutorial/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ChatGLMTokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
input_ids = tokenizer(["""
You are an assistant for question-answering tasks. Use the retrieved {context} and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
History = {history}
Question= {question}
Don't add text above in the result.
Helpful Answer:
"""], return_tensors="pt")
input_ids['input_ids'].shape[1]

100

In [16]:
input_ids

{'input_ids': tensor([[64789, 30910,    13,  3651,   383,   284,  9319,   332,  2021, 30941,
         64717,  1521,  7760, 30930,  5165,   267, 29604,   729, 11355, 30983,
           293,  3238,   267,  2021,   388,   267,   934, 30930,    13,  4906,
           344,   883, 30953, 30912,   683,   267,  3238,   680,  1105,   344,
           292,   430,   683,   284,   480,   430,  1961,   289,   794,   541,
           267,  3238,  5030,  1961,   289,   792,  2726, 16969,   974,   289,
          3238, 30930,  6936,   267,  3238,   362, 30130,   362,  1809, 30930,
            13, 21441,   542,   729, 19973, 30983,    13,  5792, 23483, 30980,
           729, 22980, 30983,    13,  6676, 30953, 30912,   784,  2254,  2129,
           291,   267,  1169, 30930,    13, 28658,   759, 20115, 30954,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1

In [4]:
from bigdl.llm.transformers import (
                AutoModel,
                AutoModelForCausalLM,
                # AutoModelForSeq2SeqLM,
            )

model = AutoModelForCausalLM.from_pretrained('/data/chatglm3-6B/', load_in_4bit=True)


You are using a model of type chatglm3 to instantiate a model of type chatglm. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████| 7/7 [00:06<00:00,  1.07it/s]
2024-01-21 13:57:55,545 - INFO - Converting the current model to sym_int4 format......


In [21]:
from transformers import TextStreamer,TextIteratorStreamer
import threading
# input_ids = tokenizer([custom_prompt_template], return_tensors="pt")
input_length = input_ids['input_ids'].shape[1]
# streamer = TextIteratorStreamer(tokenizer, timeout=60, skip_prompt=True, skip_special_tokens=True)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
# output = model.generate(**input_ids, streamer=streamer, max_new_tokens=200)
generate_kwargs = {
    "input_ids": input_ids['input_ids'],
    "attention_mask": input_ids['attention_mask'],
    "streamer" : streamer,
    "max_new_tokens": 200
}
t = threading.Thread(target=model.generate, kwargs=generate_kwargs)
# text = tokenizer.batch_decode(output[:, input_length:], skip_special_tokens=True)[0]

t.start()
history = []
for new_token in streamer.token_cache:
    history += new_token

history


[]


History:
{history}

Question:
{question}

Answer:
{answer}

{additional_info}

Answer: {answer}

{additional_info}

Answer: {answer}

{additional_info}

Answer: {answer}

{additional_info}

Answer: {answer}

{additional_info}

Answer: {answer}

{additional_info}

Answer: {answer}

{additional_info}

Answer: {answer}

{additional_info}

Answer: {answer}

{additional_info}

Answer: {answer}

{additional_info}

An


In [6]:
streamer

In [8]:
history = []
for new_token in streamer.token_cache:
    if new_token != '':
            history += new_token
            
history


[]

In [22]:
history

[]